Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.59310525 0.27060080 0.0042785742 0.0043293411 1000
 beta   0.90610755 0.04240225 0.0006704384 0.0007025419 1000
sigma   5.10693767 0.18647663 0.0029484544 0.0025522338 1000

Quantiles:
          2.5%       25.0%      50.0%       75.0%       97.5%   
alpha 154.0590000 154.410000 154.5985000 154.7770000 155.1130250
 beta   0.8234462   0.877872   0.9055715   0.9343475   0.9894761
sigma   4.7656592   4.977950   5.1005300   5.2280650   5.4959558



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.838 0.924356 5.27758; 154.922 0.870842 4.89516; … ; 154.924 0.866807 5.26418; 154.956 0.89873 5.1466]

Union{Missing, Float64}[154.774 0.916355 5.2585; 154.899 0.909099 5.23665; … ; 154.924 0.981052 5.21716; 154.372 0.843237 5.05921]

Union{Missing, Float64}[154.232 0.902484 5.1095; 154.354 0.946439 5.03633; … ; 154.337 0.911497 5.23816; 155.028 0.790026 5.16866]

Union{Missing, Float64}[154.667 0.892894 5.1336; 154.941 0.925316 5.14736; … ; 154.604 0.910227 5.0983; 154.632 0.904919 5.64354]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.59310525 0.27060080 0.0042785742 0.0043293411 1000
 beta   0.90610755 0.04240225 0.0006704384 0.0007025419 1000
sigma   5.10693767 0.18647663 0.0029484544 0.0025522338 1000

Quantiles:
          2.5%       25.0%      50.0%       75.0%       97.5%   
alpha 154.0590000 154.410000 154.5985000 154.7770000 155.1130250
 beta   0.8234462   0.877872   0.9055715   0.9343475   0.9894761
sigma   4.7656592   4.977950   5.1005300   5.2280650   5.4959558



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*